In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve > output.log 2>&1 &

In [3]:
!ollama pull phi4-mini:3.8b

In [4]:
!ollama list

NAME              ID              SIZE      MODIFIED               
phi4-mini:3.8b    78fad5d182a7    2.5 GB    Less than a second ago    


In [5]:
!pip install ollama

In [6]:
!pip install langchain langchain-community ollama

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [7]:
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory

In [8]:
# Mock knowledge base
BOOK_DATABASE = {
    "mystery": "The Hound of the Baskervilles by Arthur Conan Doyle, 200 pages, classic detective story.",
    "sci-fi": "Dune by Frank Herbert, 500 pages, epic space adventure.",
    "romance": "Pride and Prejudice by Jane Austen, 300 pages, timeless love story."
}

# Tool Functions
def book_database(query: str) -> str:
  for key, value in BOOK_DATABASE.items():
    if key in query.lower():
      return value
  return "No matching books found, but I can suggest something else!"

def reading_time_calculator(pages: str) -> str:
  try:
    pages = int(pages)
    speed = 50
    hours = pages/speed
    return f"It takes about {hours:.1f} hours to read {pages} pages at {speed} pages/speed"
  except:
    return "Error: Invalid input for calculation"

#Define tools
tools = [
    Tool(
        name="Book_Database",
        func=book_database,
        description="Used for book recommendations by genre or details"
    ),
    Tool(
        name="Reading_Time_Calculator",
        func=reading_time_calculator,
        description="Calculate reading time based on page count. Page count input taken as integer"
    )
]

# Setup LLM and Memory
llm = ChatOllama(model="phi4-mini:3.8b", temperature=0.7)
memory = ConversationBufferMemory(memory_key="chat_history")

#initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

/tmp/ipython-input-99807201.py:39: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="phi4-mini:3.8b", temperature=0.7)
/tmp/ipython-input-99807201.py:40: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
/tmp/ipython-input-99807201.py:43: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-t

### Test it

In [9]:
query="Suggest a mystery book"
response = agent.run(f"As a friendly librarian answer: {query}")
print(f"Q: {query}\nResponse: {response}")

/tmp/ipython-input-991774914.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(f"As a friendly librarian answer: {query}")




> Entering new AgentExecutor chain...
Thought: Yes

Action: Book_Database
Action Input: Genre = Mystery, Recommendation count=1
Observation: The Hound of the Baskervilles by Arthur Conan Doyle, 200 pages, classic detective story.
Thought:AI: I suggest you check out "The Hound of the Baskervilles" written by Sir Arthur Conan Doyle. It's a quintessential mystery novel with an intriguing plot about Sherlock Holmes and Dr. Watson investigating mysterious occurrences on Dartmoor in England.

Would this be something you'd enjoy? If so, it's available for lending at your local library or can also typically found as e-books through major platforms like Amazon Kindle Unlimited if you prefer digital reading!

> Finished chain.
Q: Suggest a mystery book
Response: I suggest you check out "The Hound of the Baskervilles" written by Sir Arthur Conan Doyle. It's a quintessential mystery novel with an intriguing plot about Sherlock Holmes and Dr. Watson investigating mysterious occurrences on Dartmoo

In [10]:
query="How long would it take to read 200 pages?"
response = agent.run(f"As a friendly librarian answer: {query}")
print(f"Q: {query}\nResponse: {response}")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Reading_Time_Calculator
Action Input: Calculate the average time needed for an adult reader (typically around 250-300 words per page) with reading speed of about 275 words per minute.
Observation: Error: Invalid input for calculation
Thought:It appears there was some confusion in my previous message. I apologize and will correct this by providing a direct answer to your question without using the tool.

AI: Reading time can vary based on several factors including individual reading speed, comprehension level, distractions present during reading sessions as well as breaks taken while progressing through pages or chapters of books which might affect one's pace when continuing with subsequent sections. For someone who reads at an average rate (approximately 275 words per minute), the estimated total amount for completing a book containing roughly around 200-250 pages would typically range from approximat